# Import libraries

In [1]:
import cv2
import mediapipe as mp
import numpy as np
from cvzone.HandTrackingModule import HandDetector
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import pandas as pd
import tkinter as tk
from tkinter import ttk
import random

# User interface

In [2]:
class DetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🙅🏻‍♂️ Pixel Pioneers ")
        self.root.geometry("800x600")
        self.root.configure(bg='#0d1b2a')

        self.label = ttk.Label(root, text="Choose and enjoy!", font=('Lobster', 36), background='#fca311',
                               foreground='#14213d')
        self.label.pack(pady=30)

        self.face_button = ttk.Button(root, text="👤 Face Detection", command=self.face_detection, style='TButton')
        self.face_button.pack(pady=20)

        self.color_button = ttk.Button(root, text="🤓 Color Detection", command=self.color_detection, style='TButton')
        self.color_button.pack(pady=20)

        self.volume_button = ttk.Button(root, text="🔊 Volume Control", command=self.volume_control, style='TButton')
        self.volume_button.pack(pady=20)

        self.keyboard_button = ttk.Button(root, text="⌨️ Virtual Keyboard", command=self.virtual_keyboard,
                                          style='TButton')
        self.keyboard_button.pack(pady=20)

        style = ttk.Style()
        style.configure('TButton', font=('Copperplate Gothic Bold', 24), foreground='#000000', background='#d90429',
                        padding=20, width=20)

    def face_detection(self):
        self.root.withdraw()
        add_face_detection_content(self)

    def color_detection(self):
        self.root.withdraw()
        add_color_detection_content(self)

    def volume_control(self):
        self.root.withdraw()
        add_volume_control_content(self)

    def virtual_keyboard(self):
        self.root.withdraw()
        add_virtual_keyboard_content(self)

    def show_colorful_animation(self, message, title):
        animation_window = tk.Toplevel(self.root)
        animation_window.title(title)
        animation_window.geometry("600x400")

        canvas = tk.Canvas(animation_window, bg='#023047', width=600, height=400)
        canvas.pack()

        num_ovals = 5

        def create_oval():
            x = random.randint(50, 550)
            y = random.randint(50, 350)
            radius_x = random.randint(20, 50)
            radius_y = random.randint(20, 50)
            color = "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255),
                                                  random.randint(0, 255))

            canvas.create_oval(x - radius_x, y - radius_y, x + radius_x, y + radius_y, fill=color,
                               outline='black')

        def update_animation(oval_counter=0):
            nonlocal num_ovals
            if oval_counter < num_ovals:
                create_oval()
                animation_window.after(500, update_animation, oval_counter + 1)
            else:
                animation_window.destroy()
                self.root.deiconify()

        update_animation()

# Face detection

In [3]:
def add_face_detection_content(main_app):
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:
        ret, frame = cap.read()

        if ret:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 5)

            cv2.imshow("Face Detection", frame)

        key = cv2.waitKey(1)
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    main_app.root.deiconify()

# Colour detection 

In [4]:
def add_color_detection_content(main_app):
    video_capture = cv2.VideoCapture(0)
    clicked = False
    r = g = b = x_pos = y_pos = 0

    index = ["color", "color_name", "hex", "R", "G", "B"]
    csv = pd.read_csv('colors.csv', names=index, header=None)

    def get_color_name(R, G, B):
        minimum = 10000
        for i in range(len(csv)):
            d = abs(R - int(csv.loc[i, "R"])) + abs(G - int(csv.loc[i, "G"])) + abs(B - int(csv.loc[i, "B"]))
            if d <= minimum:
                minimum = d
                cname = csv.loc[i, "color_name"]
        return cname

    def locate_function(event, x, y, flags, param):
        nonlocal b, g, r, x_pos, y_pos, clicked
        if event == cv2.EVENT_LBUTTONDBLCLK:
            clicked = True
            x_pos = x
            y_pos = y
            b, g, r = frame[y, x]
            b = int(b)
            g = int(g)
            r = int(r)

    cv2.namedWindow('Color Detection')
    cv2.setMouseCallback('Color Detection', locate_function)

    while True:
        ret, frame = video_capture.read()

        cv2.imshow("Color Detection", frame)

        if clicked:
            cv2.rectangle(frame, (20, 20), (620, 60), (b, g, r), -1)
            text = get_color_name(r, g, b) + ' R=' + str(r) + ' G=' + str(g) + ' B=' + str(b)
            cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

            if r + g + b >= 600:
                cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)

            cv2.imshow("Color Detection", frame)

        key = cv2.waitKey(1)
        if key == 27:
            break

    video_capture.release()
    cv2.destroyAllWindows()
    main_app.root.deiconify()

# 

# Volume control

In [5]:
def add_volume_control_content(main_app):
    cap = cv2.VideoCapture(0)
    detector = HandDetector(maxHands=1)

    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))

    volumeRange = volume.GetVolumeRange() 

    minVolume = volumeRange[0]
    maxVolume = volumeRange[1]

    while True:
        ret, frame = cap.read()
        hands, frame = detector.findHands(frame)
        
        cv2.rectangle(frame, (50, 180), (75, 430), (0, 255, 0), 3)
        
        if hands:
            hand = hands[0]
            lmList = hand['lmList']  
            x1, y1, _ = lmList[4]
            x2, y2, _ = lmList[8]
            
            cv2.circle(frame, (x1, y1), 10, (255, 0, 255), -1)
            cv2.circle(frame, (x2, y2), 10, (255, 0, 255), -1)
            
            centerX = (x1 + x2) // 2 
            centerY = (y1 + y2) // 2

            cv2.circle(frame, (centerX, centerY), 10, (255, 0, 255), -1)
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 255), 2)
            
            lengthOfLine = math.hypot(x2 - x1, y2 - y1)
          
            vol = np.interp(lengthOfLine, [20, 160], [minVolume, maxVolume])
            volumeBar = np.interp(lengthOfLine, [20, 160], [430, 180])
            percentage = np.interp(lengthOfLine, [20, 160], [0, 100])

            volume.SetMasterVolumeLevel(vol, None)
            
            cv2.putText(frame, f"{int(percentage)}%", (35, 160), 2, 1, (0, 0, 0), 2)
            cv2.rectangle(frame, (50, int(volumeBar)), (75, 430), (0, 255, 0), -1)
            
        cv2.imshow("frame", frame)
        
        key = cv2.waitKey(1)
        if key == 27:
            break
        
    cap.release()
    cv2.destroyAllWindows()
    main_app.root.deiconify()

# Virtual keyboard

In [6]:
def add_virtual_keyboard_content(main_app):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()

    cap = cv2.VideoCapture(0)

    keyboard_layout = [
        ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
        ["A", "S", "D", "F", "G", "H", "J", "K", "L", ""],
        ["Z", "X", "C", "V", "B", "N", "M", "", "", ""]
    ]

    keyboard_rect = (50, 50, 400, 200)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(frame_rgb)

        for i, row in enumerate(keyboard_layout):
            for j, key in enumerate(row):
                key_rect = (keyboard_rect[0] + j * 40, keyboard_rect[1] + i * 40, 40, 40)
                cv2.rectangle(frame, (key_rect[0], key_rect[1]),
                              (key_rect[0] + key_rect[2], key_rect[1] + key_rect[3]),
                              (255, 255, 255), -1)
                cv2.putText(frame, key, (key_rect[0] + 10, key_rect[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                for point in landmarks.landmark:
                    x, y, _ = map(int, (point.x * frame.shape[1], point.y * frame.shape[0], point.z * frame.shape[1]))
                    cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

                    if is_point_inside_rect((x, y), keyboard_rect):
                        key_i = (y - keyboard_rect[1]) // 40
                        key_j = (x - keyboard_rect[0]) // 40
                        if key_i < len(keyboard_layout) and key_j < len(keyboard_layout[key_i]) and keyboard_layout[
                            key_i][key_j] != "":
                            print("Pressed key:", keyboard_layout[key_i][key_j])

        cv2.imshow("Virtual Keyboard", frame)

        if cv2.waitKey(1) == 27:
            break
        def is_point_inside_rect(point, rect):
            x, y, w, h = rect
            return x < point[0] < x + w and y < point[1] < y + h

    cap.release()
    cv2.destroyAllWindows()
    main_app.root.deiconify()

In [7]:
if __name__ == "__main__":
    root = tk.Tk()
    app = DetectionApp(root)
    root.mainloop()

Pressed key: X
Pressed key: C
Pressed key: D
Pressed key: R
Pressed key: Z
Pressed key: S
Pressed key: W
Pressed key: W
Pressed key: A
Pressed key: A
Pressed key: C
Pressed key: C
Pressed key: D
Pressed key: Z
Pressed key: S
Pressed key: S
Pressed key: A
Pressed key: A
Pressed key: V
Pressed key: X
Pressed key: D
Pressed key: E
Pressed key: E
Pressed key: Z
Pressed key: S
Pressed key: W
Pressed key: W
Pressed key: Q
Pressed key: Q
Pressed key: A
Pressed key: V
Pressed key: X
Pressed key: D
Pressed key: E
Pressed key: A
Pressed key: W
Pressed key: W
Pressed key: W
Pressed key: A
Pressed key: Q
Pressed key: Q
Pressed key: Q
Pressed key: C
Pressed key: X
Pressed key: D
Pressed key: E
Pressed key: E
Pressed key: Z
Pressed key: W
Pressed key: W
Pressed key: W
Pressed key: A
Pressed key: Q
Pressed key: Q
Pressed key: A
Pressed key: C
Pressed key: X
Pressed key: D
Pressed key: E
Pressed key: E
Pressed key: Z
Pressed key: W
Pressed key: W
Pressed key: W
Pressed key: A
Pressed key: Q
Pressed ke